## Load Datasets

In [ ]:
import pandas as pd
import os
from IPython.display import display, HTML
import dask.dataframe as dd
import re
import janitor

In [ ]:
import warnings

# Suppress warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)
warnings.simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning)
warnings.simplefilter(action="ignore", category=DeprecationWarning)

In [ ]:
# Define the path 
patent_dataset = pd.read_csv(
    "C:\\Users\\shiri\\OneDrive\\Documents\\Python\\ml-projects\\data-606\\pharmaceutical-drug-price-prediction\\data\\interim\\patent_dataset.csv"
)

amp_dataset = pd.read_csv(
    "C:\\Users\\shiri\\OneDrive\\Documents\\Python\\ml-projects\\data-606\\pharmaceutical-drug-price-prediction\\data\\interim\\amp_dataset.csv"
)

drug_dataset = pd.read_csv(
    "C:\\Users\\shiri\\OneDrive\\Documents\\Python\\ml-projects\\data-606\\pharmaceutical-drug-price-prediction\\data\\interim\\drug_dataset.csv"
)
inflation = pd.read_csv(
    "C:\\Users\\shiri\\OneDrive\\Documents\\Python\\ml-projects\\data-606\\pharmaceutical-drug-price-prediction\\data\\external\\Inflation_rate.csv"
)

<ipython-input-5-f9da7c5a799d>:9: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  amp_dataset = pd.read_csv(amp_dataset)


## Basic Exploration

In [ ]:
patent_dataset.head()

,appl_no,product_no,ingredient,trade_name,strength,dosage,route,te_code,type,approval_date,patent_expire_date,applicant,applicant_full_name
0,215328,1,budesonide,budesonide,2mg/actuation,"aerosol, foam",rectal,ab,rx,2023-04-12,NaN,padagis israel,padagis israel pharmaceuticals ltd
1,205613,1,budesonide,uceris,2mg/actuation,"aerosol, foam",rectal,ab,rx,2014-10-07,NaN,salix,salix pharmaceuticals inc
2,212379,1,minocycline hydrochloride,amzeeq,eq 4% base,"aerosol, foam",topical,NaN,rx,2019-10-18,2030-10-01,journey,journey medical corp
3,212379,1,minocycline hydrochloride,amzeeq,eq 4% base,"aerosol, foam",topical,NaN,rx,2019-10-18,2037-09-08,journey,journey medical corp
4,210928,1,azelaic acid,azelaic acid,15%,"aerosol, foam",topical,NaN,discn,2020-10-07,NaN,teva pharms usa,teva pharmaceuticals usa inc


In [ ]:
amp_dataset.head()

,product_group,ingredient,strength,dosage,route,mdr_unit_type,weighted_average_of_amps,aca_ful,package_size,ndc,year,month
0,700,furosemide,80mg,tablet,oral,tab,0.039469,0.069071,100.0,63304-0626-01,2018,12
1,9591,candesartan cilexetil,8mg,tablet,oral,tab,2.489034,4.355810,90.0,49884-0659-09,2018,12
2,2421,lamotrigine,200mg,tablet,oral,tab,0.400614,0.701075,500.0,68382-0010-05,2018,12
3,4219,losartan/hydrochlorothiazide,100mg-25mg,tablet,oral,tab,0.051319,0.089808,90.0,60505-2917-09,2018,12
4,354,clonidine hydrochloride,0.3mg,tablet,oral,tab,0.018478,0.056520,1.0,51079-0301-01,2018,12


In [ ]:
drug_dataset.head()

,brnd_name,gnrc_name,tot_mftr,tot_spndng,tot_dsg_unts,tot_clms,tot_benes,avg_spnd_per_dsg_unt_wghtd,avg_spnd_per_clm,avg_spnd_per_bene,outlier_flag,year
0,1st tier unifine pentips,"pen needle, diabetic",1,167193.78,761658.0,6538.0,2341.0,0.219785,25.572619,71.419812,0.0,2018
1,1st tier unifine pentips,"pen needle, diabetic",1,139201.68,642471.0,5392.0,1878.0,0.216788,25.816335,74.122300,0.0,2019
2,1st tier unifine pentips,"pen needle, diabetic",1,118923.24,547006.0,4457.0,1595.0,0.217701,26.682351,74.560025,0.0,2020
3,1st tier unifine pentips,"pen needle, diabetic",1,102280.76,459384.0,3708.0,1313.0,0.223001,27.583808,77.898522,0.0,2021
4,1st tier unifine pentips,"pen needle, diabetic",1,70039.61,310304.0,2501.0,1147.0,0.225874,28.004642,61.063304,0.0,2022


In [ ]:
inflation.head()

,Date,Monthly Inflation Rate (seasonally adjusted),Annual Inflation Rate (not seasonally adjusted),Month,Year
0,Feb-25,0.2,2.8,2,2025
1,Jan-25,0.5,3.0,1,2025
2,Dec-24,0.4,2.9,12,2024
3,Nov-24,0.3,2.7,11,2024
4,Oct-24,0.2,2.6,10,2024


In [ ]:
print(patent_dataset.shape)
print(amp_dataset.shape)
print(drug_dataset.shape)
print(inflation.shape)

print(patent_dataset.info())
print(amp_dataset.info())
print(drug_dataset.info())
print(inflation.info())

(52251, 13)
(1057557, 12)
(16206, 12)
(133, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52251 entries, 0 to 52250
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   appl_no              52251 non-null  int64 
 1   product_no           52251 non-null  int64 
 2   ingredient           52251 non-null  object
 3   trade_name           52251 non-null  object
 4   strength             52186 non-null  object
 5   dosage               52250 non-null  object
 6   route                52214 non-null  object
 7   te_code              21643 non-null  object
 8   type                 52251 non-null  object
 9   approval_date        46380 non-null  object
 10  patent_expire_date   8473 non-null   object
 11  applicant            52251 non-null  object
 12  applicant_full_name  52251 non-null  object
dtypes: int64(2), object(11)
memory usage: 5.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105

## Data Integration

### Merge amp_dataset and drug_dataset

In [ ]:
# Get unique drug names from both datasets
unique_ingredients = amp_dataset["ingredient"].dropna().unique()
unique_generic_names = drug_dataset["gnrc_name"].dropna().unique()

# Find the intersection of both sets (drugs present in both datasets)
common_drugs = set(unique_ingredients).intersection(set(unique_generic_names))

# Print the common drugs
print(list(common_drugs)[:10])

['furosemide', 'imiquimod', 'fenofibrate nanocrystallized', 'sumatriptan succinate', 'febuxostat', 'indomethacin', 'oxaprozin', 'lamivudine', 'alprazolam', 'risperidone']


In [ ]:
# Overall count (total number of drug entries)
print(f"Total number of drug entries: {len(common_drugs)}")

Total number of drug entries: 415


There are 415 total drug entries, and some drugs appear in both datasets—such as furosemide, alprazolam, and risperidone—indicating that we can merge them for our further analysis.

In [ ]:
# Create unique identifiers by combining relevant columns in both datasets
amp_dataset["unique_key"] = (
    amp_dataset["ingredient"].astype(str) + "_" + amp_dataset["year"].astype(str)
)

drug_dataset["unique_key"] = (
    drug_dataset["gnrc_name"].astype(str) + "_" + drug_dataset["year"].astype(str)
)

# Convert to sets for efficient lookup
amp_unique = set(amp_dataset["unique_key"].dropna().unique())
drug_unique = set(drug_dataset["unique_key"].dropna().unique())

# Find the matching unique values
matching_unique_values = amp_unique.intersection(drug_unique)

# Count of matching unique values
matching_count = len(matching_unique_values)

print(f"Count of matching unique values: {matching_count}")

Count of matching unique values: 1786


In [ ]:
# Group by ingredient and get sorted list of unique strengths
unique_strengths_per_drug = (
    amp_dataset.groupby('ingredient')['strength']
    .unique()
    .reset_index(name='unique_strengths')
)

# Optional: sort strengths inside each list (alphabetically)
unique_strengths_per_drug['unique_strengths'] = unique_strengths_per_drug['unique_strengths'].apply(sorted)

# Display
unique_strengths_per_drug.head(10) 

,ingredient,unique_strengths
0,abacavir sulfate,[300mg]
1,abacavir sulfate/lamivudine,[600-300mg]
2,abiraterone acetate,[250mg]
3,acamprosate calcium,[333 mg]
4,acarbose,"[100mg, 25mg, 50mg]"
5,acebutolol hydrochloride,[200mg]
6,acetaminophen,[650 mg]
7,acetaminophen with codeine,[120-12mg/5]
8,acetaminophen; butalbital,[325mg;50mg]
9,acetaminophen; butalbital; caffeine,[325mg;50mg;40mg]


In [ ]:
unique_strengths_per_drug.shape

(609, 2)

In [ ]:
def extract_strength(s):
    try:
        return float(re.search(r"[\d\.]+", s).group())
    except:
        return None

amp_dataset['strength_num'] = amp_dataset['strength'].apply(extract_strength)

In [ ]:
max_strengths = (
    amp_dataset.groupby(['ingredient', 'year'])['strength_num']
    .transform('max')
)
amp_dataset['strength_ratio'] = amp_dataset['strength_num'] / max_strengths

We looked at all the different strength values for each drug and created a list of unique strengths for every ingredient. Then, we calculated how strong each version is compared to the highest strength available for that drug in the same year. This helps us compare drug dosages more easily across products.

In [ ]:
drug_summary = (
    drug_dataset
    .groupby(['gnrc_name', 'year'], as_index=False)
    .agg({
        'tot_mftr': 'sum',
        'tot_spndng': 'sum',
        'tot_clms': 'sum',
        'tot_benes': 'sum',
        'tot_dsg_unts': 'sum',
        'avg_spnd_per_dsg_unt_wghtd': 'mean',
        'avg_spnd_per_clm': 'mean',
        'avg_spnd_per_bene': 'mean',
    })
.round({
        'tot_mftr': 0,
        'tot_spndng': 2,
        'tot_dsg_unts': 0,
        'tot_clms': 0,
        'tot_benes': 0,
        'avg_spnd_per_dsg_unt_wghtd': 4,
        'avg_spnd_per_clm': 2,
        'avg_spnd_per_bene': 2
    })
)

In [ ]:
# Convert pandas DataFrames to Dask DataFrames
amp_dask = dd.from_pandas(amp_dataset, npartitions=25)
drug_dask = dd.from_pandas(drug_summary, npartitions=15)

# Perform an inner join on 'ingredient' and 'year'
merged_dataset_dask = amp_dask.merge(
    drug_dask,
    left_on=["ingredient", "year"],
    right_on=["gnrc_name", "year"],
    how="inner",
)

# Convert the resulting Dask DataFrame to a Pandas DataFrame (use compute)
merged_dataset = merged_dataset_dask.compute()

# Display the first few rows
merged_dataset.head()

,product_group,ingredient,strength,dosage,route,mdr_unit_type,weighted_average_of_amps,aca_ful,package_size,ndc,...,strength_ratio,gnrc_name,tot_mftr,tot_spndng,tot_clms,tot_benes,tot_dsg_unts,avg_spnd_per_dsg_unt_wghtd,avg_spnd_per_clm,avg_spnd_per_bene
0,3867,irbesartan/hydrochlorothiazide,150-12.5mg,tablet,oral,tab,0.260389,0.455681,30.0,00024-5855-30,...,0.5,irbesartan/hydrochlorothiazide,8,20170502.59,592499.0,145787.0,44197753.0,4.5255,312.2,1318.07
1,3867,irbesartan/hydrochlorothiazide,150-12.5mg,tablet,oral,tab,0.260389,0.455681,30.0,00054-0254-13,...,0.5,irbesartan/hydrochlorothiazide,8,20170502.59,592499.0,145787.0,44197753.0,4.5255,312.2,1318.07
2,3867,irbesartan/hydrochlorothiazide,150-12.5mg,tablet,oral,tab,0.260389,0.455681,90.0,00054-0254-22,...,0.5,irbesartan/hydrochlorothiazide,8,20170502.59,592499.0,145787.0,44197753.0,4.5255,312.2,1318.07
3,3867,irbesartan/hydrochlorothiazide,150-12.5mg,tablet,oral,tab,0.260389,0.455681,30.0,00093-8238-56,...,0.5,irbesartan/hydrochlorothiazide,8,20170502.59,592499.0,145787.0,44197753.0,4.5255,312.2,1318.07
4,3867,irbesartan/hydrochlorothiazide,150-12.5mg,tablet,oral,tab,0.260389,0.455681,90.0,00093-8238-98,...,0.5,irbesartan/hydrochlorothiazide,8,20170502.59,592499.0,145787.0,44197753.0,4.5255,312.2,1318.07


The two datasets were successfully merged using common columns: `ingredient` from the AMP data and `gnrc_name` from the drug data, along with `year`.

In [ ]:
merged_dataset = merged_dataset.drop(columns=['unique_key'], errors='ignore')

In [ ]:
merged_dataset.shape

(726104, 23)

In [ ]:
merged_dataset.nunique()

,0
product_group,1147
ingredient,415
strength,335
dosage,86
route,13
mdr_unit_type,7
weighted_average_of_amps,48725
aca_ful,45888
package_size,114
ndc,32903


There are 415 different drugs in the data. These drugs come in 335 different strengths, 86 dosage forms, and are taken in 13 different ways (routes). The data covers 5 years and includes monthly details. It also shows spending, claims, and usage information for each drug, with a wide range of values, suggesting good variety and depth in the dataset.

In [ ]:
# Rename drug_dataset to final_drug_dataset
drug_amp_dataset = merged_dataset.copy()

In [ ]:
# Define the path to save the CSV file
repo_path = r"C:\Users\shiri\OneDrive\Documents\Python\ml-projects\data-606\pharmaceutical-drug-price-prediction"
save_path = os.path.join(repo_path, "data", "interim")

# Ensure the directory exists
os.makedirs(save_path, exist_ok=True)

# Save the dataset as a CSV file
csv_file_path = os.path.join(save_path, "drug_amp_dataset.csv")
drug_amp_dataset.to_csv(csv_file_path, index=False)

print(f"Dataset saved successfully at: {csv_file_path}")

Dataset saved successfully at: /content/drive/My Drive/DATA_606/data/drug_amp_dataset.csv


### Merge merged_dataset and patent_dataset

In [ ]:
# Get the first 10 ingredients from amp_dataset
drug_amp_ingredients = drug_amp_dataset["ingredient"].dropna().unique()
drug_amp_ingredients_sorted = sorted(drug_amp_ingredients)[:10]

# Get the first 10 ingredients from patent_dataset
patent_ingredients = patent_dataset["ingredient"].dropna().unique()
patent_ingredients_sorted = sorted(patent_ingredients)[:10]

# Output the results
drug_amp_ingredients_sorted, patent_ingredients_sorted

(['abacavir sulfate',
  'abacavir sulfate/lamivudine',
  'abiraterone acetate',
  'acamprosate calcium',
  'acarbose',
  'acetaminophen',
  'acetaminophen with codeine',
  'acetazolamide',
  'acitretin',
  'acyclovir'],
 ['abacavir sulfate',
  'abacavir sulfate; dolutegravir sodium; lamivudine',
  'abacavir sulfate; lamivudine',
  'abacavir sulfate; lamivudine; zidovudine',
  'abaloparatide',
  'abametapir',
  'abarelix',
  'abemaciclib',
  'abiraterone acetate',
  'abiraterone acetate; niraparib tosylate'])

In [ ]:
# Get unique ingredients from both datasets
patent_ingredients = set(patent_dataset["ingredient"].dropna().unique())
drug_amp_ingredients = set(drug_amp_dataset["ingredient"].dropna().unique())

# Find common ingredients
common_ingredients = sorted(patent_ingredients.intersection(drug_amp_ingredients))

# Get the first 10 matching ingredients
matching_ingredients = common_ingredients[:10]

# Output the results
matching_ingredients

['abacavir sulfate',
 'abiraterone acetate',
 'acamprosate calcium',
 'acarbose',
 'acetaminophen',
 'acetazolamide',
 'acitretin',
 'acyclovir',
 'adapalene',
 'adefovir dipivoxil']

The merged drug and AMP dataset shares several common ingredients with the patent dataset, such as **abacavir sulfate** and **abiraterone acetate**. This overlap shows that the datasets can be successfully merged for extended analysis involving patent-related features.

In [ ]:
patent_dataset['strength_num'] = patent_dataset['strength'].apply(extract_strength)

In [ ]:
max_strength_info = (
    drug_amp_dataset.groupby('ingredient')['strength_num']
    .max()
    .reset_index()
    .rename(columns={'strength_num': 'max_strength'})
)

In [ ]:
patent_dataset = patent_dataset.merge(
    max_strength_info,
    on=['ingredient'],
    how='left'
)

patent_dataset['strength_ratio'] = (
    patent_dataset['strength_num'] / patent_dataset['max_strength']
)


We identified the unique strength values for each drug and calculated how each strength compares to the maximum strength for that drug. This helps in comparing drug dosages more easily.

In [ ]:
# Create unique identifier by combining columns
patent_dataset["unique_key"] = (
    patent_dataset["ingredient"]
    + "_"
    + patent_dataset["dosage"].astype(str)
    + "_"
    + patent_dataset["route"].astype(str)
)
drug_amp_dataset["unique_key"] = (
    drug_amp_dataset["ingredient"]
    + "_"
    + drug_amp_dataset["dosage"].astype(str)
    + "_"
    + drug_amp_dataset["route"].astype(str)
)

# Convert to sets for efficient lookup
patent_unique = set(patent_dataset["unique_key"].dropna().unique())
drug_amp_unique = set(drug_amp_dataset["unique_key"].dropna().unique())

# Find the matching unique values
matching_unique_values = patent_unique.intersection(drug_amp_unique)

# Count of matching unique values
matching_count = len(matching_unique_values)

print(f"Count of matching unique values: {matching_count}")


Count of matching unique values: 398


In [ ]:
# Drop duplicates in patent_dataset on key drug identifiers
patent_cleaned = patent_dataset.drop_duplicates(subset=["ingredient", "dosage", "route"])

# Convert to Dask
drug_amp_dask = dd.from_pandas(drug_amp_dataset, npartitions=15)
patent_dask = dd.from_pandas(patent_cleaned, npartitions=2)

# Perform a left join (safe, keeps drug_amp as base)
merged_dask1 = drug_amp_dask.merge(
    patent_dask,
    on=["ingredient", "dosage", "route"],
    how="left"
)

# Compute final result
drug_patent_dataset = merged_dask1.compute()

# Confirm no duplication occurred
print("Final merged rows:", len(drug_patent_dataset))
print("Expected (original) rows:", len(drug_amp_dataset))

Final merged rows: 726104
Expected (original) rows: 726104


We merged the cleaned **patent dataset** with the **drug_amp dataset** using common fields like `ingredient`, `dosage`, and `route`. This added relevant patent information to each drug entry without creating any duplicates.

In [ ]:
drug_patent_dataset.head()

,product_group,ingredient,strength_x,dosage,route,mdr_unit_type,weighted_average_of_amps,aca_ful,package_size,ndc,...,te_code,type,approval_date,patent_expire_date,applicant,applicant_full_name,strength_num_y,max_strength,strength_ratio_y,unique_key_y
0,3236,olanzapine,7.5mg,tablet,oral,tab,0.103186,0.180576,30.0,62756055383,...,<NA>,discn,2016-08-30,<NA>,ajanta pharma ltd,ajanta pharma ltd,2.5,20.0,0.125,olanzapine_tablet_oral
1,2095,fluconazole,150mg,tablet,oral,tab,0.468832,1.072880,1.0,16714069210,...,ab,rx,1990-01-29,<NA>,pfizer,pfizer central research,50.0,200.0,0.250,fluconazole_tablet_oral
2,3232,olanzapine,20mg,tablet,oral,tab,0.587350,1.027863,30.0,33342007207,...,<NA>,discn,2016-08-30,<NA>,ajanta pharma ltd,ajanta pharma ltd,2.5,20.0,0.125,olanzapine_tablet_oral
3,3971,esomeprazole magnesium,40 mg,capsule dr,oral,cap,2.103675,3.681431,100.0,60687024601,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>
4,7807,paliperidone,6 mg,tab er 24,oral,tab,11.300854,19.776495,30.0,50458055101,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>


In [ ]:
drug_patent_dataset.columns

Index(['product_group', 'ingredient', 'strength_x', 'dosage', 'route',
       'mdr_unit_type', 'weighted_average_of_amps', 'aca_ful', 'package_size',
       'ndc', 'year', 'month', 'strength_num_x', 'strength_ratio_x',
       'gnrc_name', 'tot_mftr', 'tot_spndng', 'tot_clms', 'tot_benes',
       'tot_dsg_unts', 'avg_spnd_per_dsg_unt_wghtd', 'avg_spnd_per_clm',
       'avg_spnd_per_bene', 'unique_key_x', 'appl_no', 'product_no',
       'trade_name', 'strength_y', 'te_code', 'type', 'approval_date',
       'patent_expire_date', 'applicant', 'applicant_full_name',
       'strength_num_y', 'max_strength', 'strength_ratio_y', 'unique_key_y'],
      dtype='object')

In [ ]:
columns_to_drop = [
    'strength_num_x', 'strength_ratio_x',
    'unique_key_x', 'strength_y', 'strength_num_y',
    'strength_ratio_y', 'unique_key_y', 'max_strength'
]

drug_patent_dataset = drug_patent_dataset.drop(columns=columns_to_drop)

# Rename strength_x to strength
drug_patent_dataset = drug_patent_dataset.rename(columns={'strength_x': 'strength'})

In [ ]:
drug_patent_dataset.shape

(726104, 30)

In [ ]:
drug_patent_dataset.isnull().sum()

,0
product_group,0
ingredient,0
strength,0
dosage,0
route,0
mdr_unit_type,0
weighted_average_of_amps,0
aca_ful,0
package_size,0
ndc,0


There are no missing values in the main drug-related columns, which means the dataset is clean and reliable for analysis. However, patent-related fields like application numbers, approval dates, and expiry dates have many missing values, indicating that not all drugs have complete patent information available.

In [ ]:
drug_patent_dataset = drug_patent_dataset.dropna(subset=['approval_date'])

In [ ]:
drug_patent_dataset.nunique()

,0
product_group,716
ingredient,274
strength,164
dosage,27
route,9
mdr_unit_type,6
weighted_average_of_amps,31906
aca_ful,30440
package_size,86
ndc,23547


The filtered or smaller subset of dataset contains fewer unique entries across most fields compared to the full dataset. There are 274 unique drugs, with key spending and claims data available for over 1,100 entries. However, patent details like expiration dates and approval information remain limited, with only a few complete records.

As per FDA/CDER SBIA Chronicles and USPTO guidelines, drug patents typically last for 20 years from the date of application or approval. In our dataset, when explicit patent expiration dates were missing, we estimated them by adding 20 years to the approval date. This helps maintain consistency with regulatory standards for patent duration.

In [ ]:
# Ensure both columns are in datetime format
drug_patent_dataset['approval_date'] = pd.to_datetime(drug_patent_dataset['approval_date'], errors='coerce')
drug_patent_dataset['patent_expire_date'] = pd.to_datetime(drug_patent_dataset['patent_expire_date'], errors='coerce')

# Fill missing patent_expire_date by adding 20 years to approval_date
drug_patent_dataset['patent_expire_date'] = drug_patent_dataset.apply(
    lambda row: row['approval_date'] + pd.DateOffset(years=20) if pd.isna(row['patent_expire_date']) else row['patent_expire_date'],
    axis=1
)

In [ ]:
drug_patent_dataset.head()

,product_group,ingredient,strength,dosage,route,mdr_unit_type,weighted_average_of_amps,aca_ful,package_size,ndc,...,avg_spnd_per_bene,appl_no,product_no,trade_name,te_code,type,approval_date,patent_expire_date,applicant,applicant_full_name
0,3236,olanzapine,7.5mg,tablet,oral,tab,0.103186,0.180576,30.0,62756055383,...,5328.33,206711.0,1.0,olanzapine,<NA>,discn,2016-08-30,2036-08-30,ajanta pharma ltd,ajanta pharma ltd
1,2095,fluconazole,150mg,tablet,oral,tab,0.468832,1.072880,1.0,16714069210,...,686.64,19949.0,1.0,diflucan,ab,rx,1990-01-29,2010-01-29,pfizer,pfizer central research
2,3232,olanzapine,20mg,tablet,oral,tab,0.587350,1.027863,30.0,33342007207,...,5328.33,206711.0,1.0,olanzapine,<NA>,discn,2016-08-30,2036-08-30,ajanta pharma ltd,ajanta pharma ltd
5,1953,gabapentin,100mg,capsule,oral,cap,0.024458,0.042802,500.0,71093-0120-05,...,3423.83,75350.0,1.0,gabapentin,ab,rx,2003-09-12,2023-09-12,actavis elizabeth,actavis elizabeth llc
6,2276,azithromycin,500mg,tablet,oral,tab,0.677082,1.410490,30.0,50268-0104-13,...,359.74,211791.0,1.0,azithromycin,ab,rx,2020-01-28,2040-01-28,alembic,alembic pharmaceuticals ltd


### Merge inflation data with merged_dataset

In [ ]:
inflation = inflation.clean_names()

In [ ]:
inflation.head()

,date,monthly_inflation_rate_seasonally_adjusted_,annual_inflation_rate_not_seasonally_adjusted_,month,year
0,Feb-25,0.2,2.8,2,2025
1,Jan-25,0.5,3.0,1,2025
2,Dec-24,0.4,2.9,12,2024
3,Nov-24,0.3,2.7,11,2024
4,Oct-24,0.2,2.6,10,2024


In [ ]:
# Ensure year and month are integers in both datasets
drug_patent_dataset[["year", "month"]] = drug_patent_dataset[["year", "month"]].astype(int)
inflation[["year", "month"]] = inflation[["year", "month"]].astype(int)

# Merge datasets on 'year' and 'month'
drug_patent_dataset = drug_patent_dataset.merge(
    inflation[["year", "month", "annual_inflation_rate_not_seasonally_adjusted_"]],
    on=["year", "month"],
    how="left",
)

# Rename the inflation column in eda_data
drug_patent_dataset.rename(
    columns={"annual_inflation_rate_not_seasonally_adjusted_": "inflation_rate"},
    inplace=True,
)

We merged the inflation data with the drug dataset using the year and month columns. This allows us to analyze drug prices while considering inflation rates for more accurate comparisons over time.

In [ ]:
selected_columns = [
    'ingredient', 'dosage', 'route', 'strength', 'mdr_unit_type',
    'weighted_average_of_amps', 'aca_ful', 'package_size', 'ndc', 'year',
    'month', 'gnrc_name', 'approval_date', 'patent_expire_date', 'inflation_rate'
]

filtered_df = drug_patent_dataset[selected_columns]
filtered_df.head()

,ingredient,dosage,route,strength,mdr_unit_type,weighted_average_of_amps,aca_ful,package_size,ndc,year,month,gnrc_name,approval_date,patent_expire_date,inflation_rate
0,olanzapine,tablet,oral,7.5mg,tab,0.103186,0.180576,30.0,62756055383,2018,1,olanzapine,2016-08-30,2036-08-30,2.1
1,fluconazole,tablet,oral,150mg,tab,0.468832,1.072880,1.0,16714069210,2018,11,fluconazole,1990-01-29,2010-01-29,2.2
2,olanzapine,tablet,oral,20mg,tab,0.587350,1.027863,30.0,33342007207,2018,5,olanzapine,2016-08-30,2036-08-30,2.8
3,gabapentin,capsule,oral,100mg,cap,0.024458,0.042802,500.0,71093-0120-05,2019,5,gabapentin,2003-09-12,2023-09-12,1.8
4,azithromycin,tablet,oral,500mg,tab,0.677082,1.410490,30.0,50268-0104-13,2018,12,azithromycin,2020-01-28,2040-01-28,1.9


## Final Dataset

In [ ]:
# Define the path to save the CSV file
repo_path = r"C:\Users\shiri\OneDrive\Documents\Python\ml-projects\data-606\pharmaceutical-drug-price-prediction"
save_path = os.path.join(repo_path, "data", "interim")

# Ensure the directory exists
os.makedirs(save_path, exist_ok=True)

# Save the dataset as a CSV file
csv_file_path = os.path.join(save_path, "drug_patent_dataset.csv")
drug_patent_dataset.to_csv(csv_file_path, index=False)

print(f"Dataset saved successfully at: {csv_file_path}")

Dataset saved successfully at: /content/drive/My Drive/DATA_606/data/drug_patent_dataset.csv
